In [1]:
from import_data_utils import *
sub = 2
cutims, voxels_ROIs_full = import_data_for_sub(sub, path)

net_size = 224
batch_size = 64
net_lim = (len(cutims) // batch_size) * batch_size

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/dtypes.py:585: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# ECoG
with open('/home/mor/ECoG/subjects_data/RDMv_zscore_dist10.pkl', 'rb') as f:
    RDMv= pkl.load(f)

with open('/home/mor/ECoG/subjects_data/all_images.pkl', 'rb') as f:
    cutims= pkl.load(f)

# RDMv, all_images
cutims = cv2.normalize(cutims, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
cutims = cutims.astype(np.uint8)

# Ecog
batch_size = 50
net_lim = (len(cutims) // batch_size) * batch_size

In [ ]:
gpu_n = 1
# allocating memory and creating the Keras net
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[gpu_n], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[gpu_n], True)

In [ ]:
# image resizing and padding
def resize_padding(x, des_size):
    """
    This function takes the image which is already in the desired size, but minimizes it to a smaller size
    and add padding to keep the original size.  
    """
    if x.shape[0] < des_size:
        raise ValueError("Desired size must be smaller than original size!")
    neutral_color = x[0,0,0]  #pixel from the image surrounding
    padding = np.ones(x.shape) * neutral_color
    start =(x.shape[0] - des_size)//2
    end = start + des_size
    resized_im = keras.layers.Resizing(des_size, des_size, interpolation='bilinear', crop_to_aspect_ratio=False,)(x).numpy()
    padding[start:end, start:end] = resized_im
    return padding

In [ ]:
#zoomin 
def zoomin_img(img, zoomsize):
    """
    This function will zoom in the image by increasing it to a bigger size and then cut and use only the middle square of it 
    to enter the network.
    
    img: an image which is already the desired size!
    zoomin = the new bigger size which we will zoom into
    """
    origin_sz = img.shape[0]
    cut = (zoomsize-origin_sz)//2
    newim = keras.layers.Resizing(zommin, zommin, interpolation='bilinear', crop_to_aspect_ratio=False,)(img).numpy()
    cutnewim = newim[cut:origin_sz+cut,cut:origin_sz+cut]
    return cutnewim

### examples of pictures filtering

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(16, 8))
im1_sub1= cutims[3]
axs[0].imshow(im1_sub1, vmin=0, vmax=255)
axs[0].set_title("Original presented image")

#example of image cutting
# axs[1].imshow(im1_sub1, vmin=0, vmax=255)

#example of image blurring
im1_blur = cv2.blur(im1_sub1,(7,7))
axs[1].imshow(im1_blur.astype(int))
axs[1].set_title("Blurred image")

# example of the resizing the picture and adding paddings.
img = keras.layers.Resizing(224, 224, interpolation='bilinear', crop_to_aspect_ratio=False,)(im1_sub1).numpy()
new_size = 100
img_res = resize_padding(img, new_size)
axs[2].imshow(img_res.astype(int), vmin=0, vmax=255)
axs[2].set_title("minimized image")

#zoomin 
a = img
zommin = int(224*1.6)
origin_sz = 224
# cut = (zommin-origin_sz)//2
# a1 = keras.layers.Resizing(zommin, zommin, interpolation='bilinear', crop_to_aspect_ratio=False,)(a).numpy()
# a2 = a1[cut:zommin-cut,cut:zommin-cut]

a2 = zoomin_img(img,zommin)
axs[3].imshow(a2.astype(int), vmin=0, vmax=255)
axs[3].set_title("maximized image")

# print(a2.shape)
# fig1, axs1 = plt.subplots(1,2, figsize=(12, 6))
# axs1[0].imshow(img.astype(int), vmin=0, vmax=255)
# axs1[1].imshow(a2.astype(int), vmin=0, vmax=255)

### ROIs

In [ ]:
# Example Voxels Bold signal from voxels_vis_ROIS V1 ventral
fig, ax = plt.subplots(figsize=(12, 5))
ax.set(xlabel="Voxel", ylabel="Stimulus")
heatmap = ax.imshow(voxels_ROIs_full['prf-visualrois']['V1v'],
                    aspect="auto", vmin=-1, vmax=1, cmap="bwr")
fig.colorbar(heatmap, shrink=.5, label="Response amplitude (Z)")
fig.tight_layout()
fig.show()

In [ ]:
# calculate RDM for fmri data
RDM_fmri = {}
for area in voxels_ROIs_full.keys():
    RDM_fmri[area] = {}
    for roi in voxels_ROIs_full[area].keys():
        RDM_fmri[area][roi] = 1-np.corrcoef(voxels_ROIs_full[area][roi][:net_lim,:])
    print("Created RDM for: " + str(area))

In [2]:
RDM_fmri

NameError: name 'RDM_fmri' is not defined

# 

# Applying VGG16

In [ ]:
class pic_edit():
    """
    This class should ease applying filters and edits to the cut squered pictue before they are being proccessed by the net. 
    It aims to contain all present and future potential dynamic properties of a picture.
    """
    def __init__(self, size = 224, blur = 1):
        self.size = size
        self.blur = blur
        
    def __repr__(self):
        return f'pic({self.size, self.blur})'
    
    def __hash__(self):
        return hash((self.size, self.blur))
    
d = {pic_edit(224,1) :1}


In [ ]:
# Bulilding Net
net = keras.applications.VGG16(input_shape=(net_size, net_size, 3),
                                include_top=True, weights='imagenet')

def preprocss_net(input_size, cut_ims, pic_params):
    """
    This function exectute all needed and optional steps before implementing the network over the data
    it resizes the input, and when needed it decreses and creates padding
    """ 
    input_len = len(cut_ims)
    input_net = np.zeros((input_len, input_size, input_size,3))
    for i in range(input_len):       
        input_net[i] = keras.layers.Resizing(input_size, input_size, interpolation='bilinear', crop_to_aspect_ratio=False,)(cut_ims[i]).numpy()
        if pic_params.blur > 0:
            input_net[i] = cv2.blur(input_net[i], (pic_params.blur, pic_params.blur))
        if pic_params.size < input_size:
            input_net[i] = resize_padding(input_net[i], pic_params.size)
        elif pic_params.size > input_size:
            cut = (pic_params.size-net_size)//2
            zoomin = keras.layers.Resizing(pic_params.size, pic_params.size, interpolation='bilinear', crop_to_aspect_ratio=False,)(input_net[i]).numpy()
            cutzoom = zoomin[cut:net_size+cut,cut:net_size+cut]
            if i < 1:
                plt.imshow(cutzoom.astype(int), vmin=0, vmax=255)
                plt.show()
            input_net[i] = cutzoom
    input_net = input_net[:net_lim,...]
    input_net = input_net.astype(int)
    return keras.applications.vgg16.preprocess_input(input_net)


In [ ]:
# CNN applying
def preds_from_batches(layer_to_eval,data,batch_size = 256, sample_size=-1 ):
    """
    Sasha's function for CNN application. Processing each layer and saving the output
    """
    data_len = data.shape[0]
    samples_set_flag = False
    for batch in range(data_len//batch_size):
        start = batch * batch_size
        end = min((batch + 1) * batch_size, data_len)
        result = layer_to_eval(data[start:end]).numpy().reshape([end-start,-1])
        if not samples_set_flag and sample_size > 0:
            samples = np.random.choice(result.shape[-1], sample_size)
            samples_set_flag = True
        if batch == 0:
            ll_out = np.zeros([data_len,result.shape[-1]]) if sample_size == -1 else np.zeros([data_len,sample_size])
        ll_out[start:end,:] = result if sample_size == -1 else result[:,samples]
    return ll_out


In [ ]:
def corr_list_extractor(RDM_net, RDM_fmri, corr_fun=spearmanr):
    """
    This function takes all RDMs from all network layers, and all RDMS from FMRI voxels and calculate correlations.
    """
    corr_dict = {}
    for brain_area in RDM_fmri.keys():
        corr_dict[brain_area] = []
        for cnn_layer in range(len(RDM_net)):
            sz=RDM_net[cnn_layer].shape[0]
            mask = np.triu(np.ones([sz,sz],dtype=np.bool),1)
            corr, pval = corr_fun(np.nan_to_num(RDM_net[cnn_layer][mask]),RDM_fmri[brain_area][mask])
            corr_dict[brain_area].append(corr)
    print ("done correlations ")
    return corr_dict

In [3]:
# RDM of network layers
# params_list = [pic_edit(net_size,0), pic_edit(int(net_size*0.5),0), pic_edit(int(net_size*1.5),0), pic_edit(int(net_size*2),0),  pic_edit(net_size,5)]
params_list = [pic_edit(net_size,0)]

RDM_net_sz={}
for params in params_list:
    RDM_net_sz[params] = {}
    print("applying network for images with params: \r\n image size:" + str(params.size) + " || blur filter size: "+ str(params.blur))
    input_net = preprocss_net(net_size, cutims, params)
    print(input_net.shape)
    for ll, layer in enumerate(net.layers):
            print(ll, layer.name)
            feature_extractor = keras.Model(inputs=net.inputs,outputs=layer.output,)
            activity = preds_from_batches(feature_extractor, input_net,batch_size=batch_size, sample_size=3000)
            RDM_net_sz[params][ll] = 1-np.corrcoef(activity)
            del feature_extractor

NameError: name 'pic_edit' is not defined

In [ ]:
cutims.shape

this is the level of the execution of the network. we can run the network as many times as we wish, changing the parameters of the pictures as desired.

In [ ]:
# RDM_net_sz

# plots

In [4]:
corr_dict = {sz : {key : corr_list_extractor(RDM_net_sz[sz], RDM_fmri[key]) for key in RDM_fmri.keys()} for sz in RDM_net_sz.keys()}

NameError: name 'RDM_net_sz' is not defined

In [ ]:
corr_dict_ecog = {sz : corr_list_extractor(RDM_net_sz[sz], RDMv) for sz in RDM_net_sz.keys()}

In [ ]:
corr_dict_ecog

In [5]:
corr_dict[list(corr_dict.keys())[0]].keys()

NameError: name 'corr_dict' is not defined

In [ ]:
colors = [axs[0].get_lines()[i].get_color() for i in range(len(list(axs[0].get_lines())))]
colors

# bootstrap

In [ ]:
def resample(mat,mat_):
    ii = np.random.randint(mat.shape[0],size=(mat.shape[0]))
    not_diag = np.eye(*mat.shape)<1e-5
    return mat[:,ii][ii,:],mat_[:,ii][ii,:],not_diag[:,ii][ii,:]

In [ ]:
def corr_list_extractor_resampled(RDM_net,RDM_fmri,corr_fun=spearmanr):
    """
    fOR BOOTSTRAP This function takes all RDMs from all network layers, and all RDMS from FMRI voxels and calculate correlations.
    """
    corr_dict = {}
    for brain_area in RDM_fmri.keys():
        corr_dict[brain_area] = []
        for cnn_layer in range(len(RDM_net)):
            sz=RDM_net[cnn_layer].shape[0]
            mask = np.triu(np.ones([sz,sz],dtype=np.bool),1)
            RDM1, RDM2, resample_mask = resample(RDM_net[cnn_layer],RDM_fmri[brain_area])
            mask = np.logical_and(mask, resample_mask)
            corr, pval = corr_fun(np.nan_to_num(RDM1[mask]),
                                  RDM2[mask])
            corr_dict[brain_area].append(corr)
        corr_dict[brain_area] = np.array(corr_dict[brain_area])#.reshape([2,-1])
    print ("done correlations ")
    return corr_dict

In [ ]:
n_resamples = 10
corr_dict_vis = {samp: {sz : corr_list_extractor_resampled(RDM_net_sz[sz], RDMv)  for sz in RDM_net_sz.keys()} for samp in range(n_resamples)}

In [ ]:
n_resamples = 10
corr_dict_bodies = {samp: {sz : corr_list_extractor_resampled(RDM_net_sz[sz], RDM_fmri['floc-bodies'])  for sz in RDM_net_sz.keys()} for samp in range(n_resamples)}

In [ ]:
corr_dict_vis_all= {}


for area in corr_dict_vis[0][list(corr_dict_vis[0].keys())[0]].keys():
    corr_dict_vis_all[area] = []
    for ii in range(n_resamples):
#         print(corr_dict_vis[ii][list(corr_dict_vis[0].keys())[0]][amp])
        corr_dict_vis_all[area].append(corr_dict_vis[ii][list(corr_dict_vis[0].keys())[0]][area])
    corr_dict_vis_all[area] = np.stack(corr_dict_vis_all[area])
# corr_dict_vis_all['V1v'].shape
# corr_dict_vis_all


In [6]:
corr_dict_bodies_all= {}


for area in corr_dict_bodies[0][list(corr_dict_bodies[0].keys())[0]].keys():
    corr_dict_bodies_all[area] = []
    for ii in range(n_resamples):
#         print(corr_dict_vis[ii][list(corr_dict_vis[0].keys())[0]][amp])
        corr_dict_bodies_all[area].append(corr_dict_bodies[ii][list(corr_dict_bodies[0].keys())[0]][area])
    corr_dict_bodies_all[area] = np.stack(corr_dict_bodies_all[area])
# corr_dict_vis_all


NameError: name 'corr_dict_bodies' is not defined

# Saving the results

In [ ]:
run_time = datetime.now().strftime("%Y%m%d%H%M")
run_time

path_for_save = "/home/mor/NDS_project/results_data/sub" + str(sub) + "/results_" + run_time
path_for_save = '/home/mor/ECoG/subjects_data/results/FF_' + run_time

# Check whether the specified path exists or not
if not os.path.exists(path_for_save):
   # Create a new directory because it does not exist
   os.makedirs(path_for_save)
   print("The new directory "+ path_for_save +" is created!")

# with open(path_for_save + '/corr_dict.pkl', 'wb') as handle:
#     pkl.dump(corr_dict, handle, protocol=pkl.HIGHEST_PROTOCOL)
#     print("saved corr_dict!")
    
with open(path_for_save + '/RDM_net_sz.pkl', 'wb') as handle:
    pkl.dump(RDM_net_sz, handle, protocol=pkl.HIGHEST_PROTOCOL)
    print("saved RDM_net_sz!")


with open(path_for_save + '/corr_dict_vis_all.pkl', 'wb') as handle:
    pkl.dump(corr_dict_vis_all, handle, protocol=pkl.HIGHEST_PROTOCOL)
    print("saved corr_dict_vis_all!")
        
with open(path_for_save + '/results_explained.txt', 'a') as f:
    f.write("Subject number: " + str(sub))
    f.write(input() + '\n')

In [ ]:
with open(path_for_save + '/corr_dict_bodies_all.pkl', 'wb') as handle:
    pkl.dump(corr_dict_bodies_all, handle, protocol=pkl.HIGHEST_PROTOCOL)
    print("saved corr_dict_bodies_all!")

In [ ]:
import pickle as pkl
with open("/home/mor/NDS_project/results_data/sub3/results_202304020811/RDM_net_sz.pkl", 'rb') as f:
    RDM_net_sz = pkl.load(f)
zer = RDM_net_sz[list(RDM_net_sz.keys())[0]]
zer[4].shape

In [ ]:
corr_dict_vis_all

In [ ]:
with open ('/home/mor/NDS_project/imported_datavoxels_ROIs.pkl', 'rb') as f:
    voxels_ROIs_full1 = pkl.load(f)
voxels_ROIs_full1['floc-bodies']['EBA'].shape

In [ ]:
with open(path + '/voxels_ROIs.pkl', 'rb') as f:
    voxels_ROIs_s1_original = pkl.load(f)
voxels_ROIs_s1_original.keys()

### Data categories

In [ ]:
# loding image metadata to find categories
cat_path = "/home/mor/annotations_trainval2017/annotations/"
fname = "instances_train2017"
f = open(cat_path + fname +'.json', "r")
data = json.loads(f.read())

imid = data['annotations'][0]['image_id']
imcat = data['annotations'][0]['category_id']
data.keys()
print([cat['name']  for cat in data['categories'] if cat['id'] == imcat])
# catname = [cat['name'] if cat['id'] == imcat for cat in data['categories']]
# data['categories'][0]
imid

im1= plt.imread(path+"/train2017/train2017/000000" + str(imid) +".jpg")
plt.imshow(im1)